<a href="https://colab.research.google.com/github/nabilah-afrin/recommendation_system_rokomri_books/blob/secondary/notebooks/Categories_bn_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# %cd /content/drive/MyDrive/Rokomari Recommendation Dataset
%cd /content/drive/MyDrive/Dataset/rokomari_books

/content/drive/MyDrive/Dataset/rokomari_books


In [3]:
!ls

books.csv				 rokomari_bn_books_author_corrected.csv
books_modified_1.csv			 rokomari_bn_books.csv
preprocessing_bn_books_authors.ipynb	 rokomari_books_preprocessing.ipynb
preprocessing_bn_books_categories.ipynb


In [4]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=282d5a324e79bf41860a363086b271d8e7b7be79508af663e559b8488e97a149
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:


In [5]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=c4d0bac09eb3421f47a38235248ccf7e6ecd2222aac0cc8ca004b4babaf79364
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import langdetect as ld
import googletrans as gt

# Only Bangla Books

In [7]:
# load the rokomari_bn_books.csv
df_bn = pd.read_csv("rokomari_bn_books_author_corrected.csv")

# comment mine and uncomment yours
# df_bn = pd.read_csv("/content/drive/MyDrive/Rokomari Recommendation Dataset/df_bn.csv")

In [8]:
df_bn.head(5)

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
0,লু স্যুন,পশ্চিমবঙ্গের বই: সমকালীন গল্প,9788194097303,No Ratings,No Reviews,320,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,নির্বাচিত গল্প সংকলন,bn,No,bn,bn
1,আর্কাদি গাইদার,পশ্চিমবঙ্গের বই: শিশু-কিশোর উপন্যাস,9788193860939,No Ratings,No Reviews,180,No image,ছাড়পত্র প্রকাশন (ইন্ডিয়া),0.0,No summary,তিমুর ও তার দলবল,bn,No,bn,bn
2,মতি নন্দী,পশ্চিমবঙ্গের বই: সমকালীন উপন্যাস,No ISBN,No Ratings,No Reviews,250,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,নায়কের প্রবেশ ও প্রস্থান,bn,No,bn,bn
3,উৎপলকুমার বসু,পশ্চিমবঙ্গের বই: বাংলা কবিতা,No ISBN,No Ratings,No Reviews,300,No image,ভালো বই (ইন্ডিয়া),0.0,No summary,হাঁস চলার পথ,bn,No,bn,bn
4,দিলরুবা হাসান,বয়স যখন ৪-৮: শিক্ষামূলক,No ISBN,No Ratings,No Reviews,129,No image,বৃত্তকলা একাডেমি,0.0,No summary,অনুভূতির ছোঁয়া,bn,No,bn,bn


# Useful Functions

In [9]:
def detect_language(row):
    try:
        lang = ld.detect(str(row))
    except:
        return 'unknown'
    return lang

# 2. Categories


In [10]:
df_bn['categories'].nunique()

1974

In [11]:
df_bn['categories'].value_counts().reset_index().loc[:20]

,categories,count
0,বাংলা কবিতা,8621
1,পশ্চিমবঙ্গের বই,7096
2,সমকালীন উপন্যাস,4339
3,সমকালীন গল্প,2756
4,শিশু-কিশোর গল্প,2124
5,চিরায়ত উপন্যাস,1180
6,ছড়া,937
7,পশ্চিমবঙ্গের বই: প্রবন্ধ,924
8,রোমান্টিক কবিতা,842
9,"নবি-রাসুল, সাহাবা, তাবেই ও অলি-আওলিয়া",839


In [12]:
df_bn.loc[df_bn['categories'] == "Hadith & Sunnah"]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
785,মাওলানা হামিদা পারভীন,Hadith & Sunnah,9843236270,No Ratings,No Reviews,196,https://ds.rokomari.store/rokomari110/ProductN...,Ahsan Publication,0.0,রাসূলুল্লাহ সাল্লাল্লাহু আলাইহি ওয়াসাল্লাম বল...,দারসে হাদীস ৬ষ্ঠ খন্ড,bn,Yes,bn,id
1814,আল হাফিজুল কাবীর আবু বকর আব্দুর রাজ্জাক ইবনে হ...,Hadith & Sunnah,No ISBN,2,No Review,15000,https://ds.rokomari.store/rokomari110/ProductN...,"Al Maktabatul Islami, Boirut",5.0,No summary,আল মুসান্নাফ লি আব্দির রাযযাক আস সান'আনি (১-১২...,bn,No,bn,id
1953,ইমাম মুহিউদ্দীন ইয়াহইয়া আননববী রহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,175,https://ds.rokomari.store/rokomari110/ProductN...,Alif Publications,0.0,No summary,রিয়াদুস সালেহীন (৩য় খণ্ড) শব্দার্থসহ,bn,No,bn,id
1955,ইমাম মুহিউদ্দীন ইয়াহইয়া আননববী রহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,175,https://ds.rokomari.store/rokomari110/ProductN...,Alif Publications,0.0,No summary,রিয়াদুস সালেহীন (৪র্থ খণ্ড) শব্দার্থসহ,bn,No,bn,id
1956,ইমাম মুহিউদ্দীন ইয়াহইয়া আননববী রহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,630,https://ds.rokomari.store/rokomari110/ProductN...,Alif Publications,0.0,No summary,"রিয়াদুস সালেহীন (১ম, ২য়, ৩য় ও ৪র্থ খণ্ড একত্রে...",bn,No,bn,id
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91739,হযরত মাওলানা উসমান গনী রহ,Hadith & Sunnah,No ISBN,2,1 Review,7599,https://ds.rokomari.store/rokomari110/ProductN...,Al-Kousar prokashani,5.0,No summary,নসরুল বারী শরহে সহীহ্‌ বুখারী (১ম খন্ড থেকে ১৩...,bn,No,bn,id
91875,মুফতী এনামুল হক রায়পুরী,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,1320,No image,Al-Kitabah Prokashini,0.0,No summary,"কামূসুল কুতুবিস সিত্তাহ ৫ম খন্ড, ভলিয়ম- ১ম এবং...",bn,No,bn,id
93506,আল্লামা ওলীউদ্দীন আবু আবদুল্লাহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,504,No image,Adhunik Prokasoni,0.0,মিশকাতুল মাসাবীহ হাদীস শাস্ত্র জগতে এক অনবদ্য ...,মিশকাতুল মাসাবীহ ৮ম খন্ড,bn,Yes,bn,id
93511,আল্লামা ওলীউদ্দীন আবু আবদুল্লাহ,Hadith & Sunnah,No ISBN,No Ratings,No Reviews,448,No image,Adhunik Prokasoni,0.0,মিশকাতুল মাসাবীহ হাদীস শাস্ত্র জগতে এক অনবদ্য ...,মিশকাতুল মাসাবীহ ৭ম খন্ড,bn,Yes,bn,id


## 2.1 Language variations in Categories

In [14]:
# df_bn['language_categories'] = df_bn['categories'].apply(detect_language)

In [15]:
df_bn['language_categories'].value_counts()

,count
language_categories,
bn,93948
en,2097
id,625
so,359
de,226
af,183
nl,137
fr,119
tl,80


In [ ]:
# df_bn.loc[df_bn['language_categories'] == 'de']

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,language_author,language_categories
1057,খুরশীদ সিহাব,"Dua, Dorud and Zikr",No ISBN,No Ratings,No Reviews,35,https://ds.rokomari.store/rokomari110/ProductN...,Akheri,0.0,আমি বইটি সংকলন করেছি মহাপবিত্র আল কুরআন ও হাদি...,রুহানি চিকিৎসা - পকেট সাইজ,bn,bn,de
1115,মোঃ জাহাঙ্গীর হোসেন (সম্পাদক),When 4-8: Fables,9789849003182,No Ratings,No Reviews,215,https://ds.rokomari.store/rokomari110/ProductN...,Akhor Prokashoni,0.0,,ঈশপের গল্প,bn,bn,de
1257,মোঃ জাহাঙ্গীর আলম,When 4-8: Fables,9789849003182,No Ratings,No Reviews,215,No image,Akhor Prokashoni,0.0,No summary,ঈশপের গল্প,bn,bn,de
1448,ঝিলমিল,When 4-8: Drawing,9789849613320,No Ratings,No Reviews,103,https://ds.rokomari.store/rokomari110/ProductN...,Akhorbirtto,0.0,বেবিস ফার্স্ট কালারিং ব্যাগ হল 10টি বইয়ের একট...,মাই ফাস্ট ড্রইং বুক ০৮ : টয়েস,bn,bn,de
1710,নাজমুস সাকিব,"Dua, Dorud and Zikr",9789849615804,No Ratings,No Reviews,56,https://ds.rokomari.store/rokomari110/ProductN...,Akij-Monowara Prokashoni,0.0,,ছন্দে ছন্দে আল্লাহর গুণবাচক নামসমূহ,bn,bn,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96571,মোহাম্মদ ফিরোজ মুকুল (এফএম),Spoken English,No ISBN,5,2 Reviews,255,https://ds.rokomari.store/rokomari110/ProductN...,FM Publications,3.0,No summary,এফএম মেথড দ্য বেস্ট মেথলজি অফ টিচিং স্পোকেন এ্...,bn,bn,de
96653,শাইখ ড. সাইদ আল-কাহতানি,"Dua, Dorud and Zikr",No ISBN,No Ratings,No Reviews,138,https://ds.rokomari.store/rokomari110/ProductN...,Fuldani Prokashony,0.0,🌸হিসনুল মুসলিম,হিসনুল মুসলিম,bn,bn,de
96672,সাইয়েদ আবুল হাসান আলী নদভী রহ,"Prophets-Messengers, Sahaba, Tabei & Oli-Awliya",No ISBN,No Ratings,No Reviews,132,https://ds.rokomari.store/rokomari110/ProductN...,Fuldani Prokashony,0.0,লেখকে ভূমিকা,কাসাসুন নাবিয়্যীন ১ম,bn,bn,de
96677,শাইখুল হাদীস মাওলানা যাকারিয়া রহ.,"Dua, Dorud and Zikr",No ISBN,No Ratings,No Reviews,60,https://ds.rokomari.store/rokomari110/ProductN...,Fuldani Prokashony,0.0,আমাদের অনেকের জীবনের দৈনন্দিন আমলের অন্যতম এক...,দুরূদ ও মানযিল,bn,bn,de


In [16]:
# def filter_cat(text):
#     return ':' in text

# category_counts_df = df_bn[df_bn['categories'].apply(filter_cat)]['categories'].value_counts().reset_index()

# another way to do this
category_counts_df = df_bn.loc[df_bn['categories'].str.contains(":"), 'categories'].value_counts().reset_index()

In [17]:
# category_counts_df = pd.read_csv("/content/drive/MyDrive/Rokomari Recommendation Dataset/category_counts.csv")

## 2.2 translate categories detected as other language to Bangla

In [19]:
# this is not necessary, also it's a good practice not to
# capitalize names

# category_counts_df.columns = ['Category', 'Count']
category_counts_df

,categories,count
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515
...,...,...
815,নবম ও দশম (ব্যবসায় শিক্ষা): বাংলাদেশ ও বিশ্ব প...,1
816,এইচএসসি ১ম বর্ষ: ইসলাম শিক্ষা পাঠ্য সহায়িকা,1
817,নবম ও দশম (মানবিক): ইংরেজি পাঠ্য সহায়িকা,1
818,"বইমেলা ২০১৮: ইতিহাস, ঐতিহ্য ও রাজনীতি",1


In [20]:
# category_counts_df.to_csv('category_counts.csv', index=False)

In [21]:
# df_bn['language_categories'] = df_bn['categories'].apply(detect_language)

In [23]:
# df_bn['language_categories'].value_counts()

In [24]:
df_bn.loc[df_bn['language_categories'].isin(["no", "sv", "pt", "vi", "cy"])]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
1010,কবি মমতাজ রেনু,Poem Criticism,9843226488,No Ratings,No Reviews,430,https://ds.rokomari.store/rokomari110/ProductN...,Ahsania Books,0.00,"Ilhami(Inspired) Poetess Mumtaz Renu, a profou...",এঞ্জেল অন মাই শোল্ডারস্,bn,Yes,bn,pt
1012,কবি মমতাজ রেনু,Poem Criticism,No ISBN,No Ratings,No Reviews,1720,https://ds.rokomari.store/rokomari110/ProductN...,Ahsania Books,0.00,No summary,মেরি সুরাত তেরা আয়না,bn,No,bn,pt
9953,এসেনশিয়াল পাবলিকেশন সম্পাদক,TOEFL,No ISBN,7,7 Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,4.86,No summary,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,No,bn,vi
9954,এসেনশিয়াল পাবলিকেশন সম্পাদক,TOEFL,No ISBN,No Ratings,No Reviews,226,https://ds.rokomari.store/rokomari110/ProductN...,এসেনশিয়াল পাবলিকেশন,0.00,CONTENTS,Cliffs Toefl Grand Review বাংলা ভার্সন,bn,Yes,bn,vi
11416,সোনিয়া পারভিন,Cyber Law,No ISBN,No Ratings,No Reviews,174,No image,Hasan Law Books,0.00,No summary,"সাইবার নিরাপত্তা আইন, ২০২৩",bn,No,bn,cy
14331,জয়কলি সম্পাদনা পরিষদ সম্পাদক,TOEFL,No ISBN,No Ratings,No Reviews,213,No image,জয়কলি পাবলিকেশন্স লিঃ,0.00,"সকল প্রশ্নের নির্ভুল উত্তর, সঠিক ব্যাখ্যা, প্র...",বারণ’স এন্ড ক্লিফস টোয়েফল - বাংলা অনুবাদ,bn,Yes,bn,vi
24467,মেজর শের মোহাম্মদ খান অবঃ অনুবাদক,Rhymes,978-984-97262-7-2,No Ratings,No Reviews,129,https://ds.rokomari.store/rokomari110/ProductN...,Raowa Publication,0.00,ছোটো সোনামনিদের ছড়ার বই। এই বইয়ের প্রতিটি ছড়া ...,ছােটােদর বাংলা ছড়া ও English Rhymes,bn,Yes,bn,cy
33028,বিচারপতি মোঃ আজিজুল হক,Cyber Law,9789849156499,No Ratings,No Reviews,898,https://ds.rokomari.store/rokomari110/ProductN...,Universal Book House,0.00,No summary,সাইবার ল এন্ড ক্রাইম,bn,No,bn,cy
33138,অ্যাডভোকেট জাহাঙ্গীর আলম সরকার,Cyber Law,978984892811,No Ratings,No Reviews,627,No image,Unique Law Book House,0.00,"১. সাইবার নিরাপত্তা আইন,২০২৩",সাইবার নিরাপত্তা আইনের ভাষ্য ও পর্নোগ্রাফি নিয়...,bn,Yes,bn,cy
34443,প্রফেসর মোঃ হাফিজুর রহমান,Honors,No ISBN,No Ratings,No Reviews,264,No image,Titas Publications,0.00,No summary,লিনিয়ার এলজাবরা - অনার্স ৩য় বর্ষ,bn,No,bn,no


In [25]:
df_bn.loc[df_bn['language_categories'].isin(["no", "sv", "pt", "vi", "cy"])].shape

(25, 15)

In [26]:
category_counts_df = category_counts_df.join(
    category_counts_df['categories'].str.split(':', expand=True).rename(
        columns={0: 'cat_1', 1: 'cat_2', 2: 'cat_3'}
    )
)

In [27]:
category_counts_df['categories_split'] = category_counts_df['categories'].str.split(":")

In [28]:
category_counts_df

,categories,count,cat_1,cat_2,cat_3,categories_split
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924,পশ্চিমবঙ্গের বই,প্রবন্ধ,None,"[পশ্চিমবঙ্গের বই, প্রবন্ধ]"
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694,পশ্চিমবঙ্গের বই,বাংলা কবিতা,None,"[পশ্চিমবঙ্গের বই, বাংলা কবিতা]"
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,None,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]"
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",None,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ..."
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515,পশ্চিমবঙ্গের বই,রচনাসমগ্র ও সংকলন,None,"[পশ্চিমবঙ্গের বই, রচনাসমগ্র ও সংকলন]"
...,...,...,...,...,...,...
815,নবম ও দশম (ব্যবসায় শিক্ষা): বাংলাদেশ ও বিশ্ব প...,1,নবম ও দশম (ব্যবসায় শিক্ষা),বাংলাদেশ ও বিশ্ব পরিচয় পাঠ্য সহায়িকা,None,"[নবম ও দশম (ব্যবসায় শিক্ষা), বাংলাদেশ ও বিশ্ব..."
816,এইচএসসি ১ম বর্ষ: ইসলাম শিক্ষা পাঠ্য সহায়িকা,1,এইচএসসি ১ম বর্ষ,ইসলাম শিক্ষা পাঠ্য সহায়িকা,None,"[এইচএসসি ১ম বর্ষ, ইসলাম শিক্ষা পাঠ্য সহায়িকা]"
817,নবম ও দশম (মানবিক): ইংরেজি পাঠ্য সহায়িকা,1,নবম ও দশম (মানবিক),ইংরেজি পাঠ্য সহায়িকা,None,"[নবম ও দশম (মানবিক), ইংরেজি পাঠ্য সহায়িকা]"
818,"বইমেলা ২০১৮: ইতিহাস, ঐতিহ্য ও রাজনীতি",1,বইমেলা ২০১৮,"ইতিহাস, ঐতিহ্য ও রাজনীতি",None,"[বইমেলা ২০১৮, ইতিহাস, ঐতিহ্য ও রাজনীতি]"


In [29]:
category_counts_df['category_length'] = category_counts_df['categories_split'].apply(lambda x: len(x))

In [30]:
category_counts_df.head()

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924,পশ্চিমবঙ্গের বই,প্রবন্ধ,None,"[পশ্চিমবঙ্গের বই, প্রবন্ধ]",2
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694,পশ্চিমবঙ্গের বই,বাংলা কবিতা,None,"[পশ্চিমবঙ্গের বই, বাংলা কবিতা]",2
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,None,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]",2
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",None,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ...",2
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515,পশ্চিমবঙ্গের বই,রচনাসমগ্র ও সংকলন,None,"[পশ্চিমবঙ্গের বই, রচনাসমগ্র ও সংকলন]",2


In [31]:
category_counts_df['category_length'].value_counts()

,count
category_length,
2,817
3,3


In [32]:
# to get the values where cat_3 == None

category_counts_df.loc[(category_counts_df['cat_3'].isna())]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924,পশ্চিমবঙ্গের বই,প্রবন্ধ,None,"[পশ্চিমবঙ্গের বই, প্রবন্ধ]",2
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694,পশ্চিমবঙ্গের বই,বাংলা কবিতা,None,"[পশ্চিমবঙ্গের বই, বাংলা কবিতা]",2
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,None,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]",2
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",None,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ...",2
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515,পশ্চিমবঙ্গের বই,রচনাসমগ্র ও সংকলন,None,"[পশ্চিমবঙ্গের বই, রচনাসমগ্র ও সংকলন]",2
...,...,...,...,...,...,...,...
815,নবম ও দশম (ব্যবসায় শিক্ষা): বাংলাদেশ ও বিশ্ব প...,1,নবম ও দশম (ব্যবসায় শিক্ষা),বাংলাদেশ ও বিশ্ব পরিচয় পাঠ্য সহায়িকা,None,"[নবম ও দশম (ব্যবসায় শিক্ষা), বাংলাদেশ ও বিশ্ব...",2
816,এইচএসসি ১ম বর্ষ: ইসলাম শিক্ষা পাঠ্য সহায়িকা,1,এইচএসসি ১ম বর্ষ,ইসলাম শিক্ষা পাঠ্য সহায়িকা,None,"[এইচএসসি ১ম বর্ষ, ইসলাম শিক্ষা পাঠ্য সহায়িকা]",2
817,নবম ও দশম (মানবিক): ইংরেজি পাঠ্য সহায়িকা,1,নবম ও দশম (মানবিক),ইংরেজি পাঠ্য সহায়িকা,None,"[নবম ও দশম (মানবিক), ইংরেজি পাঠ্য সহায়িকা]",2
818,"বইমেলা ২০১৮: ইতিহাস, ঐতিহ্য ও রাজনীতি",1,বইমেলা ২০১৮,"ইতিহাস, ঐতিহ্য ও রাজনীতি",None,"[বইমেলা ২০১৮, ইতিহাস, ঐতিহ্য ও রাজনীতি]",2


`Fill the `None` values with empty ("") string`

In [42]:
category_counts_df.loc[:, 'cat_3'].fillna("", inplace=True)

In [43]:
category_counts_df.head()

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
0,পশ্চিমবঙ্গের বই: প্রবন্ধ,924,পশ্চিমবঙ্গের বই,প্রবন্ধ,,"[পশ্চিমবঙ্গের বই, প্রবন্ধ]",2
1,পশ্চিমবঙ্গের বই: বাংলা কবিতা,694,পশ্চিমবঙ্গের বই,বাংলা কবিতা,,"[পশ্চিমবঙ্গের বই, বাংলা কবিতা]",2
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]",2
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ...",2
4,পশ্চিমবঙ্গের বই: রচনাসমগ্র ও সংকলন,515,পশ্চিমবঙ্গের বই,রচনাসমগ্র ও সংকলন,,"[পশ্চিমবঙ্গের বই, রচনাসমগ্র ও সংকলন]",2


In [45]:
category_counts_df.tail()

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
815,নবম ও দশম (ব্যবসায় শিক্ষা): বাংলাদেশ ও বিশ্ব প...,1,নবম ও দশম (ব্যবসায় শিক্ষা),বাংলাদেশ ও বিশ্ব পরিচয় পাঠ্য সহায়িকা,,"[নবম ও দশম (ব্যবসায় শিক্ষা), বাংলাদেশ ও বিশ্ব...",2
816,এইচএসসি ১ম বর্ষ: ইসলাম শিক্ষা পাঠ্য সহায়িকা,1,এইচএসসি ১ম বর্ষ,ইসলাম শিক্ষা পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, ইসলাম শিক্ষা পাঠ্য সহায়িকা]",2
817,নবম ও দশম (মানবিক): ইংরেজি পাঠ্য সহায়িকা,1,নবম ও দশম (মানবিক),ইংরেজি পাঠ্য সহায়িকা,,"[নবম ও দশম (মানবিক), ইংরেজি পাঠ্য সহায়িকা]",2
818,"বইমেলা ২০১৮: ইতিহাস, ঐতিহ্য ও রাজনীতি",1,বইমেলা ২০১৮,"ইতিহাস, ঐতিহ্য ও রাজনীতি",,"[বইমেলা ২০১৮, ইতিহাস, ঐতিহ্য ও রাজনীতি]",2
819,ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ: অনার্স ৪র্থ বর্ষ,1,ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ,অনার্স ৪র্থ বর্ষ,,"[ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ, অনার্স ৪র্থ...",2


In [44]:
category_counts_df.loc[category_counts_df['cat_1'] == "শিশু-কিশোর"]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
3,"শিশু-কিশোর: রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও ...",521,শিশু-কিশোর,"রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ও অ্যাডভেঞ্চার",,"[শিশু-কিশোর, রহস্য, গোয়েন্দা, ভৌতিক, থ্রিলার ...",2
23,শিশু-কিশোর: বিবিধ,212,শিশু-কিশোর,বিবিধ,,"[শিশু-কিশোর, বিবিধ]",2
32,শিশু-কিশোর: বিখ্যাত ব্যক্তি ও জীবনী,148,শিশু-কিশোর,বিখ্যাত ব্যক্তি ও জীবনী,,"[শিশু-কিশোর, বিখ্যাত ব্যক্তি ও জীবনী]",2
45,"শিশু-কিশোর: রূপকথা, উপকথা ও লোককাহিনী",107,শিশু-কিশোর,"রূপকথা, উপকথা ও লোককাহিনী",,"[শিশু-কিশোর, রূপকথা, উপকথা ও লোককাহিনী]",2
50,শিশু-কিশোর: রেফারেন্স ও আত্মউন্নয়নমূলক বই,95,শিশু-কিশোর,রেফারেন্স ও আত্মউন্নয়নমূলক বই,,"[শিশু-কিশোর, রেফারেন্স ও আত্মউন্নয়নমূলক বই]",2
67,শিশু-কিশোর: রচনাসমগ্র/সংকলন,70,শিশু-কিশোর,রচনাসমগ্র/সংকলন,,"[শিশু-কিশোর, রচনাসমগ্র/সংকলন]",2
68,শিশু-কিশোর: ছবি আঁকা ও গান,69,শিশু-কিশোর,ছবি আঁকা ও গান,,"[শিশু-কিশোর, ছবি আঁকা ও গান]",2
69,শিশু-কিশোর: সাধারণ জ্ঞান,67,শিশু-কিশোর,সাধারণ জ্ঞান,,"[শিশু-কিশোর, সাধারণ জ্ঞান]",2
80,"শিশু-কিশোর: গণিত, বিজ্ঞান ও প্রযুক্তি",57,শিশু-কিশোর,"গণিত, বিজ্ঞান ও প্রযুক্তি",,"[শিশু-কিশোর, গণিত, বিজ্ঞান ও প্রযুক্তি]",2
85,শিশু-কিশোর: ভাষা আন্দোলন ও মুক্তিযুদ্ধ,54,শিশু-কিশোর,ভাষা আন্দোলন ও মুক্তিযুদ্ধ,,"[শিশু-কিশোর, ভাষা আন্দোলন ও মুক্তিযুদ্ধ]",2


In [46]:
category_counts_df.loc[category_counts_df['cat_1'] == "অনুবাদ"]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
2,অনুবাদ: আত্ম-উন্নয়ন ও মেডিটেশন,617,অনুবাদ,আত্ম-উন্নয়ন ও মেডিটেশন,,"[অনুবাদ, আত্ম-উন্নয়ন ও মেডিটেশন]",2
503,"অনুবাদ: জীবনী, স্মৃতিচারণ ও সাক্ষাৎকার",3,অনুবাদ,"জীবনী, স্মৃতিচারণ ও সাক্ষাৎকার",,"[অনুবাদ, জীবনী, স্মৃতিচারণ ও সাক্ষাৎকার]",2
550,অনুবাদ: প্রবন্ধ,2,অনুবাদ,প্রবন্ধ,,"[অনুবাদ, প্রবন্ধ]",2
590,অনুবাদ: শিশু-কিশোর,2,অনুবাদ,শিশু-কিশোর,,"[অনুবাদ, শিশু-কিশোর]",2
629,অনুবাদ: কমিকস ও নকশা,1,অনুবাদ,কমিকস ও নকশা,,"[অনুবাদ, কমিকস ও নকশা]",2
651,অনুবাদ: ইসলাম,1,অনুবাদ,ইসলাম,,"[অনুবাদ, ইসলাম]",2
657,"অনুবাদ:রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও...",1,অনুবাদ,"রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভ...",,"[অনুবাদ, রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার,...",2
666,অনুবাদ: বিবিধ ধর্মীয় বই,1,অনুবাদ,বিবিধ ধর্মীয় বই,,"[অনুবাদ, বিবিধ ধর্মীয় বই]",2
671,অনুবাদ: পৌরাণিক কাহিনী,1,অনুবাদ,পৌরাণিক কাহিনী,,"[অনুবাদ, পৌরাণিক কাহিনী]",2
697,"অনুবাদ: ব্যবসা, বিনিয়োগ ও অর্থনীতি",1,অনুবাদ,"ব্যবসা, বিনিয়োগ ও অর্থনীতি",,"[অনুবাদ, ব্যবসা, বিনিয়োগ ও অর্থনীতি]",2


In [50]:
category_counts_df.loc[category_counts_df['cat_2'].str.contains("পাঠ্য সহায়িকা", regex=True)]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
100,এইচএসসি ১ম বর্ষ: গণিত পাঠ্য সহায়িকা,43,এইচএসসি ১ম বর্ষ,গণিত পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, গণিত পাঠ্য সহায়িকা]",2
110,এইচএসসি ১ম বর্ষ: পদার্থবিজ্ঞান পাঠ্য সহায়িকা,38,এইচএসসি ১ম বর্ষ,পদার্থবিজ্ঞান পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, পদার্থবিজ্ঞান পাঠ্য সহায়িকা]",2
125,এইচএসসি ১ম বর্ষ: রসায়ন পাঠ্য সহায়িকা,33,এইচএসসি ১ম বর্ষ,রসায়ন পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, রসায়ন পাঠ্য সহায়িকা]",2
181,এইচএসসি ১ম বর্ষ: জীববিজ্ঞান পাঠ্য সহায়িকা,21,এইচএসসি ১ম বর্ষ,জীববিজ্ঞান পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, জীববিজ্ঞান পাঠ্য সহায়িকা]",2
224,এইচএসসি ২য় বর্ষ: পদার্থবিজ্ঞান পাঠ্য সহায়িকা,16,এইচএসসি ২য় বর্ষ,পদার্থবিজ্ঞান পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, পদার্থবিজ্ঞান পাঠ্য সহায়িকা]",2
283,এইচএসসি ২য় বর্ষ: রসায়ন পাঠ্য সহায়িকা,11,এইচএসসি ২য় বর্ষ,রসায়ন পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, রসায়ন পাঠ্য সহায়িকা]",2
309,এইচএসসি ২য় বর্ষ: গণিত পাঠ্য সহায়িকা,9,এইচএসসি ২য় বর্ষ,গণিত পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, গণিত পাঠ্য সহায়িকা]",2
363,এইচএসসি ২য় বর্ষ: জীববিজ্ঞান পাঠ্য সহায়িকা,6,এইচএসসি ২য় বর্ষ,জীববিজ্ঞান পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, জীববিজ্ঞান পাঠ্য সহায়িকা]",2
406,এইচএসসি ১ম বর্ষ: তথ্য ও যোগাযোগ প্রযুক্তি পাঠ্...,5,এইচএসসি ১ম বর্ষ,তথ্য ও যোগাযোগ প্রযুক্তি পাঠ্য সহায়িকা,,"[এইচএসসি ১ম বর্ষ, তথ্য ও যোগাযোগ প্রযুক্তি পা...",2
448,এইচএসসি ২য় বর্ষ: সমাজবিজ্ঞান/সমাজকর্ম পাঠ্য সহ...,4,এইচএসসি ২য় বর্ষ,সমাজবিজ্ঞান/সমাজকর্ম পাঠ্য সহায়িকা,,"[এইচএসসি ২য় বর্ষ, সমাজবিজ্ঞান/সমাজকর্ম পাঠ্য ...",2


In [52]:
category_counts_df.loc[category_counts_df['cat_2'].str.contains("অনার্স", regex=True)]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
87,রাষ্ট্রবিজ্ঞান বিভাগ: অনার্স ১ম বর্ষ,52,রাষ্ট্রবিজ্ঞান বিভাগ,অনার্স ১ম বর্ষ,,"[রাষ্ট্রবিজ্ঞান বিভাগ, অনার্স ১ম বর্ষ]",2
105,ভূগোল বিভাগ: অনার্স ১ম বর্ষ,41,ভূগোল বিভাগ,অনার্স ১ম বর্ষ,,"[ভূগোল বিভাগ, অনার্স ১ম বর্ষ]",2
112,রাষ্ট্রবিজ্ঞান বিভাগ: অনার্স ৪র্থ বর্ষ,38,রাষ্ট্রবিজ্ঞান বিভাগ,অনার্স ৪র্থ বর্ষ,,"[রাষ্ট্রবিজ্ঞান বিভাগ, অনার্স ৪র্থ বর্ষ]",2
117,ইতিহাস বিভাগ: অনার্স ১ম বর্ষ,36,ইতিহাস বিভাগ,অনার্স ১ম বর্ষ,,"[ইতিহাস বিভাগ, অনার্স ১ম বর্ষ]",2
120,রাষ্ট্রবিজ্ঞান বিভাগ: অনার্স ২য় বর্ষ,35,রাষ্ট্রবিজ্ঞান বিভাগ,অনার্স ২য় বর্ষ,,"[রাষ্ট্রবিজ্ঞান বিভাগ, অনার্স ২য় বর্ষ]",2
...,...,...,...,...,...,...,...
780,লোক প্রশাসন বিভাগ: অনার্স ৩য় বর্ষ,1,লোক প্রশাসন বিভাগ,অনার্স ৩য় বর্ষ,,"[লোক প্রশাসন বিভাগ, অনার্স ৩য় বর্ষ]",2
784,ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ: অনার্স ৩য় বর্ষ,1,ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ,অনার্স ৩য় বর্ষ,,"[ইসলামের ইতিহাস ও সংস্কৃতি বিভাগ, অনার্স ৩য় ব...",2
797,পরিসংখ্যান বিভাগ: অনার্স ৪র্থ বর্ষ,1,পরিসংখ্যান বিভাগ,অনার্স ৪র্থ বর্ষ,,"[পরিসংখ্যান বিভাগ, অনার্স ৪র্থ বর্ষ]",2
801,পরিসংখ্যান বিভাগ: অনার্স ২য় বর্ষ,1,পরিসংখ্যান বিভাগ,অনার্স ২য় বর্ষ,,"[পরিসংখ্যান বিভাগ, অনার্স ২য় বর্ষ]",2


In [34]:
category_counts_df['cat_1'].value_counts()

,count
cat_1,
পশ্চিমবঙ্গের বই,98
এইচএসসি ১ম বর্ষ,42
এইচএসসি ২য় বর্ষ,37
শিশু-কিশোর,32
বয়স যখন ৮-১২,28
...,...
"ভর্তি, নিয়োগ ও প্রস্তুতি পরীক্ষা",1
বয়স যখন ৪-৮,1
ইতিহাসে বাংলাদেশ,1


In [35]:
category_1 = category_counts_df['cat_1'].value_counts()

In [36]:
category_1[5:41]

,count
cat_1,
বয়স যখন ১২-১৭,26
When 4-8,14
বয়স যখন ৪-৮,13
অনুবাদ,12
মুক্তিযুদ্ধের ইতিহাস,11
বয়স যখন ৪-৮,11
When 8-12,10
ব্যবস্থাপনা বিভাগ,10
উদ্ভিদবিজ্ঞান বিভাগ,9


In [37]:
category_counts_df.loc[category_counts_df['cat_1'] == "Foreign Language Books"]

,categories,count,cat_1,cat_2,cat_3,categories_split,category_length
362,Foreign Language Books: Other Foreign Books,6,Foreign Language Books,Other Foreign Books,None,"[Foreign Language Books, Other Foreign Books]",2
558,Foreign Language Books: Islamic Books,2,Foreign Language Books,Islamic Books,None,"[Foreign Language Books, Islamic Books]",2
627,Foreign Language Books: Religious Books,1,Foreign Language Books,Religious Books,None,"[Foreign Language Books, Religious Books]",2
680,"Foreign Language Books: Mathematics, Science &...",1,Foreign Language Books,"Mathematics, Science & Technology",None,"[Foreign Language Books, Mathematics, Science...",2
683,Foreign Language Books: Self-help and meditation,1,Foreign Language Books,Self-help and meditation,None,"[Foreign Language Books, Self-help and medita...",2
719,"Foreign Language Books: Professional, Journal ...",1,Foreign Language Books,"Professional, Journal and Reference",None,"[Foreign Language Books, Professional, Journa...",2


In [38]:
df_bn.loc[df_bn['categories'].str.contains(r'^Foreign Language Books:', regex=True)]

,author,categories,isbn,n_ratings,n_reviews,price,prod_img_link,publisher,rating,summary,title,language,summary_availability,language_author,language_categories
20496,মুনসি প্রেমচাঁদ,Foreign Language Books: Other Foreign Books,9798187335282,1,No Review,360,No image,Nai Sadi Book House,3.0,No summary,গোডান (হিন্দি),bn,No,bn,en
29632,এ আর কে শর্মা,Foreign Language Books: Other Foreign Books,9789383606115,1,No Review,180,https://ds.rokomari.store/rokomari110/ProductN...,Sri Sarada Book House (India),5.0,No summary,স্ট্রেন্থ ফার্স্ট গুডনেস নেক্সট,bn,No,bn,en
31080,পাওলো কোয়েলহো,Foreign Language Books: Other Foreign Books,9788183280921,No Ratings,No Reviews,405,No image,Wisdom Tree (India),0.0,No summary,জাহির (হিন্দি),bn,No,bn,en
33418,ড ভি আবদুর রহিম,"Foreign Language Books: Professional, Journal ...",No ISBN,2,No Review,7600,No image,U.K. Islamic Academy,3.0,No summary,এরাবিক কোর্স ফর ইংলিশ স্পিকিং স্টুডেন্টস (৩ভলি...,bn,No,bn,en
62024,বি কে শিবানী,Foreign Language Books: Other Foreign Books,9788182748576,1,No Review,351,https://ds.rokomari.store/rokomari110/ProductN...,Manjul Publishing House Pvt. Ltd (India),2.0,No summary,আসেম আনন্দ কি আর (হ্যাপিনেস আনলিমিটেড),bn,No,bn,en
62028,রোন্ডা বাইর্ন,Foreign Language Books: Other Foreign Books,9788183220941,1,No Review,718,https://ds.rokomari.store/rokomari110/ProductN...,Manjul Publishing House Pvt. Ltd (India),5.0,No summary,দ্যা সিক্রেট)(হিন্দি),bn,No,bn,en
66124,খান স্যার পাটনা,Foreign Language Books: Other Foreign Books,No ISBN,2,No Review,225,https://ds.rokomari.store/rokomari110/ProductN...,Kiran Prakashan,5.0,No summary,এনসিইআরটি ৩০০০ ফ্যাক্টস,bn,No,bn,en
76148,শায়খ সফিউর রহমান মোবারকপুরী,Foreign Language Books: Islamic Books,9786035000710,No Ratings,No Reviews,3960,No image,Darussalam (India),0.0,No summary,কালেকশন ফ্রম রিয়াদ উস সালেহীন,bn,No,bn,en
76156,দেভ আল কারদায়ী,Foreign Language Books: Religious Books,No ISBN,No Ratings,No Reviews,2800,No image,Darussalam (India),0.0,No summary,ডিকশনারি অব ইসলামিক টার্ম,bn,No,bn,en
94196,প্রসন্ত কুমার গিরি,"Foreign Language Books: Mathematics, Science &...",9789387162051,No Ratings,No Reviews,765,No image,Academic Publishers (Indian),0.0,No summary,ইন্ট্রোডাকশন টু স্ট্যাটিসটিক্স ইনক্লুডিং স্ট্য...,bn,No,bn,en


In [ ]:
print(category_1[category_1 == 1])

cat_1
কম্পিউটার সায়েন্স অ্যান্ড টেকনোলজি                      1
এইচএসসি মানবিক বিভাগ                                    1
Poem and Rhyme                                          1
Others                                                  1
Children & Teen Magazine                                1
বইমেলা ২০১৩                                             1
Self-Development                                        1
এনভায়রনমেন্টাল ইঞ্জিনিয়ারিং                             1
রহস্য, গোয়েন্দা, ভৌতিক, মিথ, থ্রিলার, ও অ্যাডভেঞ্চার    1
Islamic Book                                            1
রকমারি নির্বাচিত বই                                     1
বিদেশী ফটোগ্রাফি ম্যাগাজিন                              1
History                                                 1
Self-Help, Motivational and Meditation                  1
HSC and Equivalent                                      1
টেস্ট পেপারস                                            1
Social and Political Magazine                           1
HSC Sugg

In [ ]:
category_counts_df['cat_2'].value_counts()

,count
cat_2,
অনার্স ১ম বর্ষ,24
অনার্স ৩য় বর্ষ,22
অনার্স ২য় বর্ষ,22
অনার্স ৪র্থ বর্ষ,20
ডিগ্রি ১ম বর্ষ,19
...,...
ইন্টারভিউ,1
জীববিজ্ঞান পাঠ্য সহায়িকা,1
মানবিক বিভাগ পাঠ্য সহায়িকা,1


In [ ]:
category_2 = category_counts_df['cat_2'].value_counts()
print(category_2[category_2 == 1])

cat_2
 Science Department Pattho Sohayika             1
 রান্না, স্বাস্থ্য, ফ্যাশন ও পরিবার বিষয়ক বই    1
আইসিটি পাঠ্য সহায়িকা                           1
 হিসাববিজ্ঞান পাঠ্য সহায়িকা                    1
 এসএসসি ও সমমান                                 1
                                               ..
 ইন্টারভিউ                                      1
 জীববিজ্ঞান পাঠ্য সহায়িকা                      1
 মানবিক বিভাগ পাঠ্য সহায়িকা                    1
 বিবিধ প্রসঙ্গ                                  1
 ইংরেজি পাঠ্য সহায়িকা                          1
Name: count, Length: 262, dtype: int64
